### Hedging: Volatility Mismatch

Consider a short position in a European call option on a non-dividend paying stock with a maturity of one year and strike K = 99 EUR. Let the one-year risk-free interest rate be 6% and the current stock price be 100 EUR. Furthermore, assume that the volatility is 20%.

Use the Euler method to perform a hedging simulation.

In [8]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt

#### Roadmap
We are short a European call option with $K=99, T = 1$ on a stock with $S_0 = 100, r = 0.06, \sigma = 0.20$. We need to hedge this short option position to manage risk.

---

The stock evolves according to GBM, discretized via the Euler scheme. The Black-Scholes price dynamics under the risk neutral measure are:
$$
dS_t = rS_t \, dt + \sigma S_t \, dW_t
$$
Following the Euler scheme, the discretization of the model is given by:
$$
\hat{S}_{t+\Delta t} = \hat{S}_t + r \hat{S}_t \Delta t + \sigma \hat{S}_t \sqrt{\Delta t} \, Z
$$
where $\Delta t = T / N$ depends on how frequently we rebalance.

---

At each rebalancing date, the Black-Scholes call price is given by:
$$
C_{BS}(S_t, K, t, T, \sigma) = S_t N(d_+) - Ke^{-r(T-t)} N(d_-)
$$
where
$$
d_{\pm} = \frac{\ln \frac{S_t}{K} + (r \pm \frac{\sigma^2}{2})(T - t)}{\sigma \sqrt{T - t}}
$$
We hedge the short option position at time $t$ by buying $\Delta_t$ shares, where:
$$
\Delta_t = \frac{\partial C}{\partial S}(t, S_t) = N(d_+)
$$
This choice removes the linear term in $\delta S$. We reassess and rebalance at each subsequent time step $t + \Delta t$.

---

The payoff of an option can be replicated by trading in the underlying stock and a risk-free bond. Consider a portfolio consisting of $x_t$ units of the cash account (with value $B_t = e^{rt}$) and $y_t$ shares of stock. The portfolio value is:
$$
\Pi_t = x_t B_t + y_t S_t
$$
A portfolio is self-financing if changes in its value solely stem from changes in asset prices, with no external cash flows. Formally:
$$
d\Pi_t = x_t \, dB_t + y_t \, dS_t = rx_t B_t \, dt + y_t(r S_t \, dt + \sigma S_t \, dW_t)
$$
By matching the stochastic terms in the option price with the replicating portfolio, we find the hedge ratio:
$$
y_t = \frac{\partial C}{\partial S} = \Delta_t
$$
This is the delta hedge: at each instant, hold $\Delta_t$ shares of stock to eliminate exposure to small movements in $S_t$.

---

In practice, continuous rebalancing is impossible. Instead, we rebalance at discrete times $t_0, t_1, \ldots, t_N$. At time $t_i$, the portfolio consists of $\Delta_{t_i}$ shares of stock and a cash position of $\Pi_{t_i} - \Delta_{t_i} S_{t_i}$. The self-financing condition implies:
$$
\Pi_{t_{i+1}} = \Pi_{t_i} + \underbrace{(\Pi_{t_i} - \Delta_{t_i} S_{t_i}) \cdot r \Delta t}_{\text{interest on cash}} + \underbrace{\Delta_{t_i}(S_{t_{i+1}} - S_{t_i})}_{\text{stock gain/loss}}
$$
We initialize the replicating portfolio with the option premium received at time zero:
$$
\Pi_0 = C(S_0, 0)
$$
At maturity $T = t_N$, the P&L of the hedged position is the difference between the replicating portfolio value and the option payoff we must deliver:
$$
\text{P\&L} = \Pi_T - \max(S_T - K, 0)
$$
If the hedge were perfect, $\text{P\&L} = 0$ almost surely. In practice, discrete hedging introduces replication error.

---

To understand what drives this replication error, we analyze the per-step PnL. Over a short interval $\Delta t$, the PnL for the short option, long $\Delta_t$ shares position is:
$$
\text{PnL} = -[C(t + \Delta t, S_{t+\Delta t}) - C(t, S_t)] + rC(t, S_t) \Delta t + \Delta_t(\delta S - r S_t \Delta t)
$$
where $\delta S = S_{t + \Delta t} - S_t$. Expanding $C(t + \Delta t, S_{t+\Delta t})$ to second order and using $\Delta_t = \partial C / \partial S$ to cancel the linear terms, we get the carry PnL:
$$
\text{PnL} = -A(t, S_t) \Delta t - B(t, S_t) \left( \frac{\delta S}{S_t} \right)^2
$$
where
$$
A(t, S_t) = \frac{\partial C}{\partial t} - rC + r S_t \frac{\partial C}{\partial S}, \quad B(t, S_t) = \frac{1}{2} S_t^2 \frac{\partial^2 C}{\partial S^2}
$$
A natural risk management condition is to require that our hedged portfolio neither consistently gains nor loses money on average. Formally, this condition implies:
$$
A(t, S) = -\hat{\sigma}^2 B(t, S), \quad \text{for all } t, S
$$
Since $C$ is priced with $\hat{\sigma}$ and satisfies the Black-Scholes PDE, substituting yields the simplified expression:
$$
\text{PnL} = -\frac{1}{2} S^2 \frac{\partial^2 C_{\hat{\sigma}}}{\partial S^2} \left[ \left( \frac{\delta S}{S} \right)^2 - \hat{\sigma}^2 \Delta t \right]
$$
When $\sigma_{\text{real}} = \sigma_{\text{imp}} = \hat{\sigma}$, the realized squared returns satisfy $\mathbb{E}[(\delta S / S)^2] = \hat{\sigma}^2 \Delta t$, so the expected PnL vanishes. The residual variance is purely due to discrete rebalancing.

#### 1. Matching Volatility

Conduct an experiment where the volatility in the stock price process matches the volatility used in the delta computation (i.e., both set to 20%). Vary the frequency of hedge adjustments (from daily to weekly) and explain the results.

In [2]:
# Parameters

S0 = 100 # Spot price
K = 99 # strike price
T = 1.0 # maturity
r = 0.06 # risk-free rate
sigma = 0.20 # volatility (real same as implied for task 1)

def get_call_price(S, t, K, T, r, sigma):
    """Black-Scholes European call price (eq. 91)"""
    tau = T - t
    if tau <= 0:
        return max(S - K, 0)
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    d2 = d1 - sigma * np.sqrt(tau)
    return S * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2)

def get_delta(S, t, K, T, r, sigma):
    """Black Scholes delta = N(d1) (eq.97 + Greeks table)"""
    tau = T - t
    if tau <= 0:
        return 1.0 if S > K else 0.0
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * tau) / (sigma * np.sqrt(tau))
    return norm.cdf(d1)
    

In [4]:
def hedge(S0, T, K, r, sigma_r, sigma_i, N_steps, seed = None):
    """
    Simulate one delta-hedge path.
    
    sigma_r: volatility driving the stock
    sigma_i: volatility used for pricing/hedging (Black Scholes)
    
    """
    if seed is not None:
        np.random.seed(seed)
        
    dt = T / N_steps
    S = np.zeros(N_steps + 1)
    S[0] = S0
    
    # Simulate stock path via Euler scheme
    Z = np.random.randn(N_steps)
    for i in range(N_steps):
        S[i+1] = S[i] + r * S[i] * dt + sigma_r * S[i] * np.sqrt(dt) * Z[i]
        
    # Init hedge
    delta_0 = get_delta(S[0], 0, K, T, r, sigma_i)
    C0 = get_call_price(S[0], 0, K, T, r, sigma_i)
    B = C0 - delta_0 * S[0] # Bank acccount
    delta_old = delta_0
    
    # Rebalance at each step
    for i in range(1, N_steps):
        t_i = i * dt
        B = B * np.exp(r * dt)
        delta_new = get_delta(S[i], t_i, K, T, r, sigma_i)
        B = B - (delta_new - delta_old) * S[i]           # Fund rebalancing
        delta_old = delta_new
        
    # Final PnL at maturity
    B = B * np.exp(r * dt)  
    payoff = max(S[-1] - K, 0)   # Option settlement
    PnL = B + delta_old * S[-1] - payoff
    
    return PnL, S

In [7]:
# Monte Carlo across different rebalancing frequnecies
n_sims = 10_000
frequencies = {
    "Daily (N=252)": 252,
    "Every 2 days (N=126)": 126,
    "Weekly (N=52)": 52,
    "Biweekly (N=26)": 26,
}

sigma_i = 0.20
sigma_r = 0.20

results = {}
for label, N in frequencies.items():
    pnls = []
    for sim in range(n_sims):
        pnl, _ = hedge(S0, T, K, r, sigma_r, sigma_i, N, seed=sim)
        pnls.append(pnl)
    results[label] = np.array(pnls)
    
    print(f"{label}:")
    print(f"  Mean PnL:  {np.mean(pnls):.4f}")
    print(f"  Std PnL:   {np.std(pnls):.4f}")
    print()

Daily (N=252):
  Mean PnL:  -0.0054
  Std PnL:   0.4267

Every 2 days (N=126):
  Mean PnL:  -0.0106
  Std PnL:   0.6030

Weekly (N=52):
  Mean PnL:  -0.0020
  Std PnL:   0.9358

Biweekly (N=26):
  Mean PnL:  -0.0038
  Std PnL:   1.3136



#### Interpretation

---

#### 2. Mismatched Volatility

Perform numerical experiments where the volatility in the stock price process does not match the volatility used in the delta valuation. Run computational experiments for various levels of volatility and discuss the outcomes.

In [10]:
# Task 2: Mismatched Volatility
sigma_imp = 0.20
sigma_real_grid = [0.10, 0.15, 0.20, 0.25, 0.30]
N_steps = 252
n_simulations = 10_000


results_mismatch = {}
for sigma_real in sigma_real_grid:
    pnls = []
    for sim in range(n_simulations):
        pnl, _ = hedge(S0, T, K, r, sigma_real, sigma_imp, N_steps, seed=sim)
        pnls.append(pnl)
    results_mismatch[sigma_real] = np.array(pnls)
    print(f"σ_real = {sigma_real:.2f}:")
    print(f"  Mean PnL:  {np.mean(pnls):.4f}")
    print(f"  Std PnL:   {np.std(pnls):.4f}")

σ_real = 0.10:
  Mean PnL:  3.5862
  Std PnL:   1.1181
σ_real = 0.15:
  Mean PnL:  1.8635
  Std PnL:   0.7369
σ_real = 0.20:
  Mean PnL:  -0.0054
  Std PnL:   0.4267
σ_real = 0.25:
  Mean PnL:  -1.9297
  Std PnL:   1.0024
σ_real = 0.30:
  Mean PnL:  -3.8816
  Std PnL:   1.9202


#### Interpretation

---